# 日本のCOVID19感染者数分析(情報提供:NHK)

In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")

In [3]:
pref["日付"] = pd.to_datetime(pref["日付"] )

In [4]:
latest_date_str = pref["日付"].max().strftime("%Y/%m/%d")

In [5]:
all_jp = pref.copy().groupby(["日付"], as_index=False).sum()
del all_jp["都道府県コード"]
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]

In [6]:
all_jp["weekday"] = all_jp["日付"].dt.weekday
weekdays = pd.DataFrame({"曜日":["月","火","水","木","金","土","日"]})
all_jp = pd.merge(all_jp, weekdays, left_on="weekday", right_index=True, how="left")

## 移動平均

In [7]:
all_jp["感染者数移動平均7"] = all_jp["各地の感染者数_1日ごとの発表数"].rolling(7).mean()
all_jp["死者数移動平均7"] = all_jp["各地の死者数_1日ごとの発表数"].rolling(7).mean()
all_jp.tail()

,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,各地の直近1週間の人口10万人あたりの感染者数,weekday,曜日,感染者数移動平均7,死者数移動平均7
981,2022-09-23,63841,87,15484.91,4,金,59391.000000,104.142857
982,2022-09-24,39197,68,14305.11,5,土,54854.714286,94.142857
983,2022-09-25,46761,49,13691.48,6,日,52391.142857,90.142857
984,2022-09-26,43662,74,13835.16,0,月,53194.571429,90.428571
985,2022-09-27,43578,105,14395.27,1,火,54886.142857,87.285714


## 増減速度

In [8]:
all_jp["感染者数移動平均7増減"] = all_jp["感染者数移動平均7"].diff()
all_jp["死者数移動平均7増減"] = all_jp["死者数移動平均7"].diff()
all_jp.tail(7)

,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,各地の直近1週間の人口10万人あたりの感染者数,weekday,曜日,感染者数移動平均7,死者数移動平均7,感染者数移動平均7増減,死者数移動平均7増減
979,2022-09-21,69805,126,16273.45,2,水,62327.714286,126.857143,-4347.000000,-8.285714
980,2022-09-22,77359,102,15984.07,3,木,61116.857143,117.000000,-1210.857143,-9.857143
981,2022-09-23,63841,87,15484.91,4,金,59391.000000,104.142857,-1725.857143,-12.857143
982,2022-09-24,39197,68,14305.11,5,土,54854.714286,94.142857,-4536.285714,-10.000000
983,2022-09-25,46761,49,13691.48,6,日,52391.142857,90.142857,-2463.571429,-4.000000
984,2022-09-26,43662,74,13835.16,0,月,53194.571429,90.428571,803.428571,0.285714
985,2022-09-27,43578,105,14395.27,1,火,54886.142857,87.285714,1691.571429,-3.142857


### 感染者数前週同曜日比

In [9]:
import datetime
all_jp["新規感染者数先週比"] = all_jp["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)
all_jp["死者数先週比"] = all_jp["各地の死者数_1日ごとの発表数"].pct_change(periods=7)
all_jp["新規感染者数先週差"] = all_jp["各地の感染者数_1日ごとの発表数"].diff(periods=7)
all_jp["死者数先週差"] = all_jp["各地の死者数_1日ごとの発表数"].diff(periods=7)


## 都道府県別

In [10]:
pref["感染者数移動平均7"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).mean()

### 人口あたり

In [11]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)
pref = pd.merge(pref, pop, on="都道府県名", how="left")

In [12]:
pref["10万人あたり感染者数"] = pref["各地の感染者数_1日ごとの発表数"]/(pref["人口"]/100000)
pref["10万人あたり感染者数移動平均7"] = pref["10万人あたり感染者数"].rolling(7).mean()
pref["10万人あたり感染者数1週間合計"] = pref["10万人あたり感染者数"].rolling(7).sum()

### 感染者数前日比

In [13]:
import numpy as np
pref["感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
pref["感染者数前日差"] = pref["各地の感染者数_1日ごとの発表数"].diff()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"感染者数前日比"] = np.nan
pref["感染者数前日比移動平均7"] = pref["感染者数前日比"].rolling(7).mean()
pref["10万人あたり感染者数移動平均7前日差"] = pref["10万人あたり感染者数移動平均7"].diff()
pref["感染者数移動平均7前日差"] = pref["感染者数移動平均7"].diff()

### 感染者数前週同曜日比

In [14]:
import datetime
# 一番古い日付から一週間分NANを設定
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)
pref["新規感染者数先週差"] = pref["各地の感染者数_1日ごとの発表数"].diff(periods=7)
pref.loc[(pref["日付"] < pref["日付"].min()+datetime.timedelta(days=7)),"新規感染者数先週比"] = np.nan
pref.loc[(pref["日付"] < pref["日付"].min()+datetime.timedelta(days=7)),"新規感染者数先週差"] = np.nan


## 重症者数

In [15]:
severe = pd.read_csv("COVID-19-jp/severe_cases_daily.csv")
severe["Date"] = pd.to_datetime(severe["Date"])
#severe = severe[severe.Prefecture=="ALL"]
severe = severe[["Date","ALL"]].rename(columns={"ALL":"Severe cases"})

severe.tail()

,Date,Severe cases
891,2022-10-17,118
892,2022-10-18,116
893,2022-10-19,125
894,2022-10-20,113
895,2022-10-21,106


In [16]:
print(severe.count())
print(all_jp.count())

Date            896
Severe cases    896
dtype: int64
日付                         986
各地の感染者数_1日ごとの発表数           986
各地の死者数_1日ごとの発表数            986
各地の直近1週間の人口10万人あたりの感染者数    986
weekday                    986
曜日                         986
感染者数移動平均7                  980
死者数移動平均7                   980
感染者数移動平均7増減                979
死者数移動平均7増減                 979
新規感染者数先週比                  974
死者数先週比                     926
新規感染者数先週差                  979
死者数先週差                     979
dtype: int64


## 曜日別陽性者集計

In [17]:
all_jp_week_mean = all_jp.groupby(["weekday"],as_index=False).mean()[["各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数"]].sort_index()
all_jp_week_mean = pd.merge(all_jp_week_mean, weekdays, left_index=True, right_index=True, how="left")
all_jp_week_mean

,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,曜日
0,14333.588652,37.631206,月
1,20565.489362,55.297872,火
2,24677.014286,52.171429,水
3,24404.787234,49.021277,木
4,22960.468085,50.567376,金
5,22649.787234,41.957447,土
6,20534.212766,29.865248,日


In [18]:
import matplotlib.pyplot as plt
import japanize_matplotlib
#%matplotlib widget
%matplotlib notebook
#%matplotlib inline

In [19]:
fig=plt.figure(figsize=(9.5,8))
plt.subplot(2,1,1) # (rows, columns, panel number)
plt.title("国内の陽性者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp["日付"],all_jp["各地の感染者数_1日ごとの発表数"],label="陽性者")
plt.bar(severe["Date"],severe["Severe cases"],label="重症者数")
plt.plot_date(all_jp["日付"],all_jp["感染者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x",rotation=70, labelsize=9)
plt.ylabel("人")
plt.grid()
plt.legend()

plt.subplot(2,1,2) # (rows, columns, panel number)
plt.title("国内の死者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp["日付"],all_jp["各地の死者数_1日ごとの発表数"],label="死者")
plt.plot_date(all_jp["日付"],all_jp["死者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x",rotation=70, labelsize=9)
plt.grid()
plt.legend()

fig.tight_layout()


<IPython.core.display.Javascript object>

In [20]:
fig=plt.figure(figsize=(9.5,8),tight_layout=True)
plt.subplot(2,1,1) # (rows, columns, panel number)
#fig, ax = plt.subplots(2,1,figsize=(9,8),tight_layout=True)
#wp_ax = ax[0]
plt.title("国内の曜日別陽性者数平均({}時点)".format(latest_date_str))
plt.bar(all_jp_week_mean["曜日"],all_jp_week_mean["各地の感染者数_1日ごとの発表数"])
#wd_ax = ax[1]
plt.subplot(2,1,2) # (rows, columns, panel number)
plt.title("国内の曜日別平均死者数平均({})時点)".format(latest_date_str))
plt.bar(all_jp_week_mean["曜日"],all_jp_week_mean["各地の死者数_1日ごとの発表数"])

<IPython.core.display.Javascript object>

<BarContainer object of 7 artists>

## 直近の推移

In [21]:
recent_date = pd.to_datetime(datetime.datetime.now()).round("D")- datetime.timedelta(days=90)
all_jp_recent = all_jp.loc[all_jp["日付"]>=recent_date]
all_jp_recent.tail(30)

,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,各地の直近1週間の人口10万人あたりの感染者数,weekday,曜日,感染者数移動平均7,死者数移動平均7,感染者数移動平均7増減,死者数移動平均7増減,新規感染者数先週比,死者数先週比,新規感染者数先週差,死者数先週差
956,2022-08-29,95872,233,52600.43,0,月,185752.000000,282.428571,-6444.428571,-1.714286,-0.319975,-0.048980,-45111.0,-12.0
957,2022-08-30,152483,319,50502.82,1,火,177760.000000,279.000000,-7992.000000,-3.428571,-0.268411,-0.069971,-55944.0,-24.0
958,2022-08-31,169721,338,47519.74,2,水,167236.714286,284.285714,-10523.285714,5.285714,-0.302662,0.122924,-73663.0,37.0
959,2022-09-01,149837,306,44550.06,3,木,156932.571429,285.714286,-10304.142857,1.428571,-0.324955,0.033784,-72129.0,10.0
960,2022-09-02,128664,347,41812.46,4,金,147769.714286,289.000000,-9162.857143,3.285714,-0.332669,0.070988,-64140.0,23.0
961,2022-09-03,123045,288,39350.61,5,土,139623.000000,293.000000,-8146.714286,4.000000,-0.316690,0.107692,-57027.0,28.0
962,2022-09-04,107745,216,37228.56,6,日,132481.000000,292.428571,-7142.000000,-0.571429,-0.316941,-0.018182,-49994.0,-4.0
963,2022-09-05,68000,221,36049.16,0,月,128499.285714,290.714286,-3981.714286,-1.714286,-0.290721,-0.051502,-27872.0,-12.0
964,2022-09-06,112151,320,34413.49,1,火,122737.571429,290.857143,-5761.714286,0.142857,-0.264502,0.003135,-40332.0,1.0
965,2022-09-07,129747,247,32637.93,2,水,117027.000000,277.857143,-5710.571429,-13.000000,-0.235528,-0.269231,-39974.0,-91.0


In [22]:
severe_recent = severe.loc[severe["Date"]>=recent_date]
severe_recent.tail(30)

,Date,Severe cases
866,2022-09-22,262
867,2022-09-23,238
868,2022-09-24,239
869,2022-09-25,232
870,2022-09-26,225
871,2022-09-27,225
872,2022-09-28,204
873,2022-09-29,188
874,2022-09-30,178
875,2022-10-01,165


In [23]:
fig=plt.figure(figsize=(12,8))
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("国内の陽性者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp["日付"],all_jp["各地の感染者数_1日ごとの発表数"],label="陽性者(infected)")
plt.bar(severe["Date"],severe["Severe cases"],label="重症者(severe)")
plt.plot_date(all_jp["日付"],all_jp["感染者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x",rotation=70, labelsize=9)
plt.ylabel("人(people)")
plt.grid()
plt.legend(fontsize=9)

plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("国内の死者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp["日付"],all_jp["各地の死者数_1日ごとの発表数"],label="死者(death)")
plt.plot_date(all_jp["日付"],all_jp["死者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x",rotation=70, labelsize=9)
plt.grid()
plt.legend(fontsize=9)

plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("直近の国内の陽性者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp_recent["日付"],all_jp_recent["各地の感染者数_1日ごとの発表数"],label="陽性者(infected)")
plt.bar(severe_recent["Date"],severe_recent["Severe cases"],label="重症者数(severe)")
plt.plot_date(all_jp_recent["日付"],all_jp_recent["感染者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
#plt.plot_date(all_jp_latest["日付"],all_jp_latest["直近１週間の新規感染者数合計"]/10,label="直近１週間の合計(10人)", fmt="-",lw=2.0, color="m")
plt.tick_params(axis="x",labelsize=9)
plt.ylabel("人(people)")
plt.grid()
latest_date = all_jp_recent["日付"].max()
latest_value = all_jp_recent.loc[all_jp_recent["日付"]==latest_date,"各地の感染者数_1日ごとの発表数"].iloc[0]
plt.text(latest_date,latest_value ,"{:,}".format(latest_value),fontsize=8,va="bottom")
#latest_value = severe_recent.loc[severe_recent["Date"]==latest_date,"Severe cases"].iloc[0]
#plt.text(latest_date,latest_value ,"{:,}".format(latest_value),fontsize=8,va="bottom",ha="left")
plt.legend(fontsize=9)

plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("直近の国内の死者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp_recent["日付"],all_jp_recent["各地の死者数_1日ごとの発表数"],label="死者(death)")
plt.plot_date(all_jp_recent["日付"],all_jp_recent["死者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x", labelsize=9)
plt.ylabel("人(people)")
plt.grid()
latest_value = all_jp_recent.loc[all_jp_recent["日付"]==latest_date,"各地の死者数_1日ごとの発表数"].iloc[0]
plt.text(latest_date,latest_value ,"{:,}".format(latest_value),fontsize=8,va="bottom",ha="center")
plt.legend(fontsize=9)

fig.text(1,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=8,ha="right")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [24]:
fig.savefig("out/covid-nhk-domestic-daily.png")

### 都道府県別

In [25]:
prefs = {
    "東京都": ("-",1.1,""),
#    "沖縄県": ("-",1.1,""),
    "埼玉県": ("-",1.1,""),
    "千葉県": ("-",1.1,""),
    "神奈川県": ("-",1.1,""),
#    "栃木県": ("-",1.1,""),
#    "茨城県": ("-",1.1,""),
#    "群馬県": ("-",1.1,""),
#    "静岡県": ("-",1.1,""),
    "京都府": ("-",1.1,""),
    "大阪府": ("-",1.1,""),
    "兵庫県": ("-",1.1,""),
#    "福岡県": ("-",1.1,""),
#    "北海道": ("--",1.0,"2021-05-25"),
#    "石川県": ("--",1.0,""),
#    "福島県": ("--",1.0,""),
#    "愛知県": ("--",1.0,"2021-05-25"),
#    "滋賀県": ("--",1.0,""),
#    "熊本県": ("--",1.0,""),
#    "広島県": ("--",1.0,"2021-04-15"),
#    "岐阜県": ("--",1.0,"2021-05-16"),
#    "三重県": ("--",1.0,"2021-05-27"),
#    "山梨県": (":",1.0,""),
#    "佐賀県": (":",1.5,""),
#    "奈良県": (":",1.5,""),
#    "山口県": (":",1.5,""),
#    "高知県": (":",1.5,""),
#    "青森県": (":",1.5,""),
#    "富山県": (":",1.5,""),
#    "岩手県": (":",1.5,""),
#    "島根県": (":",1.5,""),
}

In [26]:
pref_recent = pref.loc[pref["日付"]>=recent_date]

In [27]:
fig = plt.figure(figsize=(9.5,7))
plt.title("主要都道府県の１０万人当たり感染者数１週間合計の推移({}時点)".format(latest_date_str))
dates = pref_recent["日付"].unique()
for p in prefs:
    values = pref_recent.loc[pref_recent["都道府県名"]==p]["10万人あたり感染者数1週間合計"]
    plt.plot_date(dates,values,label=p, fmt=prefs[p][0], lw=prefs[p][1])
    if prefs[p][2] == "":
        text_date = dates.max()
    else:
        text_date = pd.to_datetime(prefs[p][2])
    text_y = pref_recent.loc[(pref_recent["都道府県名"]==p) & (pref_recent["日付"]==text_date)]["10万人あたり感染者数1週間合計"]
    plt.text(text_date,text_y,p,fontsize=8,va="center")
plt.plot([dates.min(),dates.max()],[15,15],"-",lw=1.5,color="black")
plt.text(dates.min(),15,"Stage3",ha="right",va="center",fontsize=9)
plt.plot([dates.min(),dates.max()],[25,25],"-",lw=1.5,color="black")
plt.text(dates.min(),25,"Stage4",ha="right",va="center",fontsize=9)
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("人数（人口１０万人あたり）")
fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)

plt.tight_layout()

<IPython.core.display.Javascript object>

In [28]:
fig.savefig("out/covid-nhk-domestic-major-pref-daily-recent.png")

In [29]:
fig = plt.figure(figsize=(10,10),tight_layout=True)
fig.text(0.5,1,"全都道府県の１０万人当たり感染者数１週間合計の推移({}時点)".format(latest_date_str),va="top",ha="center")
fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)
dates = pref_recent["日付"].unique()
all_prefs = pref_recent["都道府県名"].unique()
for i,p in enumerate(all_prefs):
    plt.subplot(7,7,i+1) # (rows, columns, panel number)
    plt.title(p,fontsize=8,y=0.7)
    values = pref_recent.loc[pref_recent["都道府県名"]==p]["10万人あたり感染者数1週間合計"]
    if p in prefs:
        fmt=prefs[p][0]
        lw=prefs[p][1]
    else:
        fmt=":"
        lw=0.5
#    plt.plot_date(dates,values,fmt=fmt, lw=lw)
    plt.plot_date(dates,values,fmt="-", lw=0.5)
    plt.tick_params(axis="x",labelsize=8,rotation=90)
    plt.tick_params(axis="y",labelsize=8)
    

"""
    if prefs[p][2] == "":
        text_date = dates.max()
    else:
        text_date = pd.to_datetime(prefs[p][2])
    text_y = pref_recent.loc[(pref_recent["都道府県名"]==p) & (pref_recent["日付"]==text_date)]["10万人あたり感染者数1週間合計"]
    plt.text(text_date,text_y,p,fontsize=8,va="center")
plt.plot([dates.min(),dates.max()],[15,15],"-",lw=1.5,color="black")
plt.text(dates.min(),15,"Stage3",ha="right",va="center",fontsize=9)
plt.plot([dates.min(),dates.max()],[25,25],"-",lw=1.5,color="black")
plt.text(dates.min(),25,"Stage4",ha="right",va="center",fontsize=9)
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("人数（人口１０万人あたり）")
"""


<IPython.core.display.Javascript object>

'\n    if prefs[p][2] == "":\n        text_date = dates.max()\n    else:\n        text_date = pd.to_datetime(prefs[p][2])\n    text_y = pref_recent.loc[(pref_recent["都道府県名"]==p) & (pref_recent["日付"]==text_date)]["10万人あたり感染者数1週間合計"]\n    plt.text(text_date,text_y,p,fontsize=8,va="center")\nplt.plot([dates.min(),dates.max()],[15,15],"-",lw=1.5,color="black")\nplt.text(dates.min(),15,"Stage3",ha="right",va="center",fontsize=9)\nplt.plot([dates.min(),dates.max()],[25,25],"-",lw=1.5,color="black")\nplt.text(dates.min(),25,"Stage4",ha="right",va="center",fontsize=9)\nplt.grid()\nplt.legend(bbox_to_anchor=(0, 1),loc=\'upper left\', fontsize=9)\nplt.ylabel("人数（人口１０万人あたり）")\n'

## 感染速度

fig = plt.figure(figsize=(9.5,10),tight_layout=True)
ax1 = plt.subplot(3,1,1) # (rows, columns, panel number)
plt.title("直近の国内の感染速度（７日間移動平均の増減）({}時点)".format(latest_date_str))
plt.plot_date(all_jp_recent["日付"],all_jp_recent["感染者数移動平均7増減"],label="陽性者",fmt="-")
plt.tick_params(axis="x",labelsize=9)
plt.ylabel("陽性者数増減（人）")
plt.grid()
plt.ylim(-1500,1500)
ax2 = plt.twinx()
ax2.plot_date(all_jp_recent["日付"],all_jp_recent["死者数移動平均7増減"],label="死者",fmt="-", color="orange")
plt.ylabel("死者数増減（人）")
plt.ylim(-25,25)
h1,l1 = ax1.get_legend_handles_labels()
h2,l2 = ax2.get_legend_handles_labels()
ax2.legend(h1+h2,l1+l2)

chart_data = pref_recent
dates = chart_data["日付"].unique()

ax1 = plt.subplot(3,1,2) # (rows, columns, panel number)
plt.title("首都圏１都３県の感染速度（１０万人当たり新規感染者数７日間移動平均の増減）({}時点)".format(latest_date_str))
target_p = ["東京都","神奈川県","埼玉県","千葉県"]
#stack_values = []
for p in target_p:
#    stack_values.append(chart_data.loc[chart_data["都道府県名"]==p]["10万人あたり感染者数移動平均7前日差"])
    values = chart_data.loc[chart_data["都道府県名"]==p]["10万人あたり感染者数移動平均7前日差"]
#    plt.fill_between(dates,values)
    plt.plot_date(dates,values,label=p, fmt="-", lw=1)
#plt.stackplot(dates, stack_values, labels=target_p)
plt.plot([dates.min(),dates.max()],[0,0],lw=1,color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("１０万人あたり新規感染者数増減")

ax1 = plt.subplot(3,1,3) # (rows, columns, panel number)
plt.title("関西圏２府３県の感染速度（１０万人当たり新規感染者数７日間移動平均の増減）({}時点)".format(latest_date_str))
target_p = ["大阪府","兵庫県","奈良県","京都府","和歌山県"]
for p in target_p:
    values = chart_data.loc[chart_data["都道府県名"]==p]["10万人あたり感染者数移動平均7前日差"]
    plt.plot_date(dates,values,label=p, fmt="-", lw=1)
plt.plot([dates.min(),dates.max()],[0,0],lw=1,color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("１０万人あたり新規感染者数増減")


fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)

In [30]:
#fig.savefig("out/covid-nhk-domestic-velocity-daily.png")

## 感染速度2(前週同曜日比)

In [31]:
fig = plt.figure(figsize=(9.5,10),tight_layout=True)
ax1 = plt.subplot(3,1,1) # (rows, columns, panel number)
plt.title("直近の国内の感染速度（前週同曜日差の推移）({}時点)".format(latest_date_str))
plt.plot_date(all_jp_recent["日付"],all_jp_recent["新規感染者数先週差"],label="陽性者",fmt="-")
#plt.plot_date(all_jp["日付"],all_jp["新規感染者数先週差"],label="陽性者",fmt="-")
plt.tick_params(axis="x",labelsize=9)
plt.ylabel("陽性者数前週同曜日差(人)")
plt.grid()
plt.ylim(-35000,35000)
ax2 = plt.twinx()
ax2.plot_date(all_jp_recent["日付"],all_jp_recent["死者数先週差"],label="死者",fmt="-", color="orange")
plt.ylabel("死者数前週同曜日差(人)")
plt.ylim(-100,100)
plt.plot([dates.min(),dates.max()],[0,0],lw=1.1,color="black")
h1,l1 = ax1.get_legend_handles_labels()
h2,l2 = ax2.get_legend_handles_labels()
ax2.legend(h1+h2,l1+l2,bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)

chart_data = pref_recent
dates = chart_data["日付"].unique()

ax1 = plt.subplot(3,1,2) # (rows, columns, panel number)
plt.title("首都圏１都３県の感染速度（前週同曜日差の推移）({}時点)".format(latest_date_str))
target_p = ["東京都","神奈川県","埼玉県","千葉県"]
#stack_values = []
for p in target_p:
    values = chart_data.loc[chart_data["都道府県名"]==p]["新規感染者数先週差"]
    plt.plot_date(dates,values,label=p, fmt="-", lw=1)
plt.plot([dates.min(),dates.max()],[0,0],lw=1,color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("陽性者数前週同曜日差(人)")


ax1 = plt.subplot(3,1,3) # (rows, columns, panel number)
plt.title("関西圏２府３県の感染速度（前週同曜日差の推移）({}時点)".format(latest_date_str))
target_p = ["大阪府","兵庫県","奈良県","京都府","和歌山県"]
for p in target_p:
    values = chart_data.loc[chart_data["都道府県名"]==p]["新規感染者数先週差"]
    plt.plot_date(dates,values,label=p, fmt="-", lw=1)
plt.plot([dates.min(),dates.max()],[0,0],lw=1,color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("陽性者数前週同曜日差(人)")


fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)

<IPython.core.display.Javascript object>

Text(0, 0, '※新型コロナ関連の情報提供:NHK ／\u3000※人口の情報提供:e-Stat')

In [32]:
fig.savefig("out/covid-nhk-domestic-velocity-daily2.png")

In [33]:
fig = plt.figure(figsize=(9.5,7))
plt.title("主要都道府県の１０万人当たり感染者数({}時点・全期間)".format(latest_date_str))
dates = pref["日付"].unique()
for p in prefs:
    if (p != "東京都") :
        continue
    values = pref.loc[pref["都道府県名"]==p]["10万人あたり感染者数"]
    plt.plot_date(dates,values,label=p, fmt=prefs[p][0], lw=prefs[p][1])
    if prefs[p][2] == "":
        text_date = dates.max()
    else:
        text_date = pd.to_datetime(prefs[p][2])
    text_y = pref.loc[(pref["都道府県名"]==p) & (pref["日付"]==text_date)]["10万人あたり感染者数"]
    plt.text(text_date,text_y,round(text_y.iloc[-1],2),fontsize=8,va="center")
    plt.plot([dates.min(),dates.max()],[text_y,text_y],"-",color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("人数（人口１０万人あたり）")
fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)

plt.tight_layout()

<IPython.core.display.Javascript object>